# Married Women's Rights Reform Visualization

For this project we are going to explore data looking at married women's economic rights reform from 1835-1920 (data gathered by Sara Chatfield).
We will be exploring this data visually.
The goal is to provide a set of choropleth visualizations for each of the columns containing dates such that the resulting visualizations tell
the story by conveying through color, texture, or both the time lines of achievement of each milestone/column in the dataset (for the contiguous
48 states only.)

In [1]:
# import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import json
import folium

import matplotlib.pyplot as plt

Read in and view codebook info (i.e., readme for the dataset).

In [2]:
file_name = 'sturmcodebook.txt'
with open(file_name, 'r') as f:
    lines = f.read()
    print(lines)

Codebook
Married Womenâ€™s Economic Rights Reform, 1835-1920
Sara Chatfield (Sara.Chatfield@du.edu)

state â€“ State abbreviation

fips â€“ FIPS State Code

icpsr â€“ ICPSR State Code

debtfree â€“ Year of passage of state law protecting married womenâ€™s separate property from her husbandâ€™s debts

effectivemwpa â€“ Year of passage of state law granting married women control and management rights over their separate property

earnings â€“ Year of passage of state law granting married women ownership of their wages or earnings on par with other separate property

wills â€“ Year of passage of state law granting married women the ability to write wills without their husband's consent or other restrictions

soletrader â€“ Year of passage of state law granting married women as a class the right to sign contracts and engage in business without consent of husband

Note: This study examines the passage of married womenâ€™s economic rights reforms prior to the Nineteenth Amendment (1920, gran

In [3]:
# read the data into a pandas dataframe
sturm_data = pd.read_csv('SturmData.csv')
sturm_data.head()

,state,fips,icpsr,debtfree,effectivemwpa,earnings,wills,soletrader
0,AL,1,41,1846.0,NaN,1887.0,NaN,NaN
1,AR,5,42,1835.0,1873.0,1873.0,1868.0,1868.0
2,AZ,4,61,1864.0,1871.0,1973.0,NaN,1871.0
3,CA,6,71,1850.0,1872.0,1872.0,1874.0,1872.0
4,CO,8,62,1861.0,1861.0,1861.0,1874.0,1874.0


Notice the presence of 'NaN' values, as well as, the out of order 'fips' code column.

In [4]:
# swap rows 1 and 2, so data will line up with geojson data later
temp1, temp2 = sturm_data.iloc[2].copy(), sturm_data.iloc[1].copy()
sturm_data.iloc[1], sturm_data.iloc[2] = temp1, temp2
sturm_data.head()

,state,fips,icpsr,debtfree,effectivemwpa,earnings,wills,soletrader
0,AL,1,41,1846.0,NaN,1887.0,NaN,NaN
1,AZ,4,61,1864.0,1871.0,1973.0,NaN,1871.0
2,AR,5,42,1835.0,1873.0,1873.0,1868.0,1868.0
3,CA,6,71,1850.0,1872.0,1872.0,1874.0,1872.0
4,CO,8,62,1861.0,1861.0,1861.0,1874.0,1874.0


In [5]:
# take a look at data for FL, since according to info from dataset, we should display its entries as 'NaN'
sturm_data.loc[sturm_data['state'] == 'FL']

,state,fips,icpsr,debtfree,effectivemwpa,earnings,wills,soletrader
7,FL,12,43,1845.0,1943.0,1892.0,1823.0,NaN


In [6]:
# update FL values to 'NaN'
cols = ['debtfree', 'effectivemwpa', 'earnings', 'wills']
for col in cols:
    sturm_data.loc[sturm_data.state.eq('FL'), col] = np.nan
sturm_data.iloc[7]

state             FL
fips              12
icpsr             43
debtfree         NaN
effectivemwpa    NaN
earnings         NaN
wills            NaN
soletrader       NaN
Name: 7, dtype: object

In [7]:
# prep state geo data for map
states = gpd.read_file('states_geo.json')

In [8]:
states.head()

,id,name,density,geometry
0,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,02,Alaska,1.264,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,04,Arizona,57.050,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
3,05,Arkansas,56.430,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
4,06,California,241.700,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."


In [9]:
# drop density column as it is not needed for this project
states.drop(columns= ['density'], inplace= True)
states.head()

,id,name,geometry
0,01,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,02,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,04,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
3,05,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
4,06,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."


In [10]:
# drop the states/territories we don't need
drop_states = ['Alaska', 'Hawaii', 'District of Columbia', 'Puerto Rico']
for to_drop in drop_states:
    states.drop(states[states['name'] == to_drop].index, inplace= True)

In [11]:
# reindex
states.reset_index(drop= True, inplace= True)
states.head()

,id,name,geometry
0,01,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,04,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
2,05,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
3,06,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."
4,08,Colorado,"POLYGON ((-107.91973 41.00391, -105.72895 40.9..."


In [12]:
# sanity check to make sure datasets are matching up dimension-wise
print(len(sturm_data), len(states))

48 48


In [13]:
# add features to geopandas data
new_features = ['fips', 'debtfree', 'effectivemwpa', 'earnings', 'wills', 'soletrader']
for feature in new_features:
    states[new_features] = sturm_data[new_features]
states.head()

,id,name,geometry,fips,debtfree,effectivemwpa,earnings,wills,soletrader
0,01,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1,1846.0,NaN,1887.0,NaN,NaN
1,04,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",4,1864.0,1871.0,1973.0,NaN,1871.0
2,05,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",5,1835.0,1873.0,1873.0,1868.0,1868.0
3,06,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",6,1850.0,1872.0,1872.0,1874.0,1872.0
4,08,Colorado,"POLYGON ((-107.91973 41.00391, -105.72895 40.9...",8,1861.0,1861.0,1861.0,1874.0,1874.0


In [14]:
# visually inspecting columns data to make sure both data sets match up
sturm_data.head()

,state,fips,icpsr,debtfree,effectivemwpa,earnings,wills,soletrader
0,AL,1,41,1846.0,NaN,1887.0,NaN,NaN
1,AZ,4,61,1864.0,1871.0,1973.0,NaN,1871.0
2,AR,5,42,1835.0,1873.0,1873.0,1868.0,1868.0
3,CA,6,71,1850.0,1872.0,1872.0,1874.0,1872.0
4,CO,8,62,1861.0,1861.0,1861.0,1874.0,1874.0


In [ ]:
# convert geopandas data back to json
states_json = states.to_json()
states_json

In [ ]:
# we want a json object we can work with, not just the string representation
states2= json.loads(states_json)
states2

Now that our data is cleaned up, we can set up our choropleth maps.

In [ ]:
# set up base layer
m = folium.Map(location= [39.83, -98.58],
               tiles= None,
               overlay= False,
               zoom_start= 4)

# set up tile layer
folium.TileLayer('CartoDB Positron',
                  overlay= True,
                  control= False).add_to(m)

# set up title
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Married Women's Economic Rights Reform</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# set up choropleths for each feature
fg1 = folium.FeatureGroup(name= 'Separate Debts', overlay= False).add_to(m)
fg2 = folium.FeatureGroup(name= 'Property Rights', overlay= False).add_to(m)
fg3 = folium.FeatureGroup(name= 'Ownership of Wages/Earnings', overlay= False).add_to(m)
fg4 = folium.FeatureGroup(name= 'Ability to Write Wills', overlay= False).add_to(m)
fg5 = folium.FeatureGroup(name= 'Right to Sign Contracts/Engage in Business', overlay= False).add_to(m)

# since we are only displaying one legend, set up a year_span that will be stable across all choropleths
year_span = np.arange(1800.0, 1995.0, 15.0).tolist()

sep_debts = folium.Choropleth(geo_data= states2,
                              name= 'Separate Debts',
                              data= sturm_data,
                              columns= ['fips', 'debtfree'],
                              threshold_scale = year_span,
                              key_on= 'properties.fips',
                              fill_color= 'PuRd',
                              nan_fill_color= 'white',
                              fill_opacity= 0.7,
                              highlight= True,
                              control= True,
                              overlay= False,
                              legend_name= 'Year',
                              ).add_to(m)

prop_rights = folium.Choropleth(geo_data= states2,
                              name= 'Property Rights',
                              data= sturm_data,
                              columns= ['fips', 'effectivemwpa'],
                              threshold_scale = year_span,
                              key_on= 'properties.fips',
                              fill_color= 'PuRd',
                              nan_fill_color= 'white',
                              fill_opacity= 0.7,
                              highlight= True,
                              control= True,
                              overlay= False,
                              ).add_to(m)

earnings = folium.Choropleth(geo_data= states2,
                              name= 'Ownership of Wages/Earnings',
                              data= sturm_data,
                              columns= ['fips', 'earnings'],
                              threshold_scale = year_span,
                              key_on= 'properties.fips',
                              fill_color= 'PuRd',
                              nan_fill_color= 'white',
                              fill_opacity= 0.7,
                              highlight= True,
                              control= True,
                              overlay= False,
                              ).add_to(m)

wills = folium.Choropleth(geo_data= states2,
                              name= 'Ability to Write Wills',
                              data= sturm_data,
                              columns= ['fips', 'wills'],
                              threshold_scale = year_span,
                              key_on= 'properties.fips',
                              fill_color= 'PuRd',
                              nan_fill_color= 'white',
                              fill_opacity= 0.7,
                              highlight= True,
                              control= True,
                              overlay= False,
                              ).add_to(m)

contracts = folium.Choropleth(geo_data= states2,
                              name= 'Right to Sign Contracts/Engage in Business',
                              data= sturm_data,
                              columns= ['fips', 'soletrader'],
                              threshold_scale = year_span,
                              key_on= 'properties.fips',
                              fill_color= 'PuRd',
                              nan_fill_color= 'white',
                              fill_opacity= 0.7,
                              highlight= True,
                              control= True,
                              overlay= False,
                              ).add_to(m)

# not sure if I need the following
# seems as if I include the others, then the colors don't change when you switch between layers
sep_debts.geojson.add_to(fg1)
#prop_rights.geojson.add_to(fg2)
#earnings.geojson.add_to(fg3)
#wills.geojson.add_to(fg4)
#contracts.geojson.add_to(fg5)

# add tooltips
sep_debts.geojson.add_child(folium.features.GeoJsonTooltip(['name', 'debtfree'], aliases= ['State', 'Yr Protecting Property from Husband\'s Debts']))
prop_rights.geojson.add_child(folium.features.GeoJsonTooltip(['name', 'effectivemwpa'], aliases= ['State', 'Yr Control Over Separate Property Rights']))
earnings.geojson.add_child(folium.features.GeoJsonTooltip(['name', 'earnings'], aliases= ['State', 'Yr Ownership of Wages/Earnings']))
wills.geojson.add_child(folium.features.GeoJsonTooltip(['name', 'wills'], aliases= ['State', 'Yr Right to Write Wills w/out Husband Consent']))
contracts.geojson.add_child(folium.features.GeoJsonTooltip(['name', 'soletrader'], aliases= ['State', 'Yr Right to Engage in Business w/out Husband Consent']))

# suppress other legends
for p in [prop_rights, earnings, wills, contracts]:
    for key in p._children:
        if key.startswith('color_map'):
            del(p._children[key])

folium.LayerControl(collapsed= True).add_to(m)

In [ ]:
#show map
m

In [ ]:
# save figure
m.save('Married_Womens_Rights_Reform.html')